# 📊 Evaluasi Model SmartCash

Notebook ini menjelaskan proses evaluasi model SmartCash sesuai dengan interface CLI.

## 📋 Daftar Isi
1. [Setup Environment](#setup)
2. [Load Model & Data](#load)
3. [Evaluasi Reguler](#regular)
4. [Evaluasi Skenario Penelitian](#research)
5. [Analisis Hasil](#analysis)

## 1. Setup Environment <a id='setup'></a>

In [ ]:
import sys
import os
from pathlib import Path

# Add project root to path
project_root = Path().absolute().parent
sys.path.append(str(project_root))

# Import handlers
from smartcash.handlers.data_handler import DataHandler
from smartcash.handlers.evaluation_handler import EvaluationHandler
from smartcash.utils.metrics import calculate_metrics
from smartcash.utils.visualization import plot_results

## 2. Load Model & Data <a id='load'></a>

In [ ]:
# Load config
import yaml
from dotenv import load_dotenv

load_dotenv()

with open(project_root / 'configs' / 'base_config.yaml') as f:
    config = yaml.safe_load(f)

# Load model
model_path = project_root / 'outputs' / 'best_model.pt'
evaluator = EvaluationHandler(model_path, config)

# Load test data
handler = DataHandler()
test_loader = handler.get_test_loader(
    batch_size=config['evaluation']['batch_size']
)

## 3. Evaluasi Reguler <a id='regular'></a>

In [ ]:
# Evaluasi standar
metrics = evaluator.evaluate_regular(test_loader)

print("\nHasil Evaluasi Regular:")
print(f"Akurasi: {metrics['accuracy']:.4f}")
print(f"Precision: {metrics['precision']:.4f}")
print(f"Recall: {metrics['recall']:.4f}")
print(f"F1-Score: {metrics['f1']:.4f}")
print(f"mAP: {metrics['mAP']:.4f}")
print(f"Waktu Inferensi: {metrics['inference_time']:.1f}ms")

## 4. Evaluasi Skenario Penelitian <a id='research'></a>

In [ ]:
# Evaluasi skenario penelitian
scenarios = [
    'normal_light',
    'low_light',
    'high_light',
    'rotation',
    'partial_occlusion'
]

research_results = {}
for scenario in scenarios:
    print(f"\nEvaluasi Skenario: {scenario}")
    metrics = evaluator.evaluate_research(
        test_loader,
        scenario=scenario
    )
    research_results[scenario] = metrics
    
    print(f"mAP: {metrics['mAP']:.4f}")
    print(f"Waktu Inferensi: {metrics['inference_time']:.1f}ms")

## 5. Analisis Hasil <a id='analysis'></a>

In [ ]:
# Plot confusion matrix
evaluator.plot_confusion_matrix()

# Plot precision-recall curve
evaluator.plot_pr_curve()

# Plot perbandingan skenario
evaluator.plot_scenario_comparison(research_results)

# Save results
evaluator.save_results(
    regular_metrics=metrics,
    research_metrics=research_results,
    output_dir=project_root / 'results'
)